In [1]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

# Filter Deep Dive
This notebook introduces all the available Filters

<span style="color: #FF8C00;">==========================================================</span>

**If you find this tool useful, a sponsorship would be greatly appreciated!**

**https://github.com/sponsors/HansjoergW**

How to get in touch

* Found a bug: https://github.com/HansjoergW/sec-fincancial-statement-data-set/issues
* Have a remark: https://github.com/HansjoergW/sec-fincancial-statement-data-set/discussions/categories/general
* Have an idea: https://github.com/HansjoergW/sec-fincancial-statement-data-set/discussions/categories/ideas
* Have a question: https://github.com/HansjoergW/sec-fincancial-statement-data-set/discussions/categories/q-a
* Have something to show: https://github.com/HansjoergW/sec-fincancial-statement-data-set/discussions/categories/show-and-tell

<span style="color: #FF8C00;">==========================================================</span>

## Basics

A few basic points

* All Filters are implmented for the RawDataBag and for the JoinedDataBag. Depending for which databag type the filter is implemented its postfix is either `RawFilter` or `JoinedFilter`.
* All Filters do not copy the dataframes. They just apply filter on existing dataframes, but usually don't create new ones.
* All Filters have a `filter()` method which takes a databag as parameter and returns a new databag as parameter (again, the dataframes are not copied in the new instance of the databag).
* Instead of using the `filter()` method of a databag you can also use the index operator.
```
a_filter = USDOnlyRawFilter()
a_rawdatabag: RawDataBag = ...

# use the filter() method of the filter..
new_databag = a_filter.filter(a_rawdatabag)

# or use the filter method of the databab
new_databag = a_rawdatabag.filter(a_filter)
```
* Calls to the `filter()` method of the databag can be chained as follows
```
filter1 = USDOnlyRawFilter()
filter2 = OfficialTagsOnlyRawFilter()
a_rawdatabag: RawDataBag = ...

new_databag = a_rawdatabag.filter(filter1).filter(filter2)

```
* The index operator (`[]`) of the databag class is forwarded to the `filter()` method, therefore you can write the previous call as follows:
```
new_databag = a_rawdatabag[filter1][filter2]
```

## Load Demo Databag

We use the following dataset to demonstrate the filters.

In [2]:
from secfsdstools.e_collector.zipcollecting import ZipCollector

databag = ZipCollector.get_zip_by_name('2022q4.zip').collect()

print("sub: ", databag.sub_df.shape)
print("pre: ", databag.pre_df.shape)
print("num: ", databag.num_df.shape)

2025-02-02 06:13:13,658 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 06:13:14,372 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-02 06:13:14,377 [INFO] parallelexecution      items to process: 1
2025-02-02 06:13:14,379 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2022q4.zip
2025-02-02 06:13:17,960 [INFO] parallelexecution      commited chunk: 0


sub:  (7280, 36)
pre:  (806371, 10)
num:  (3543392, 10)


## `AdshRawFilter`

This filter lets you select the data for certain reports by their adsh number. Just provide the list of the adsh numbers you are interested in in the constructor of the filter.

It operates on all dataframes (sub, pre, and num).

In [3]:
from secfsdstools.e_filter.rawfiltering import AdshRawFilter

apple_10k_2022_adsh = "0000320193-22-000108"
adsh_filter = AdshRawFilter(adshs=[apple_10k_2022_adsh])

filtered_databag = databag[adsh_filter]

# since we filtered only a single adsh, there is only one line in the sub_df
print("sub: ", filtered_databag.sub_df.shape)
# also the pre_df and the num_df only contain data for a single report
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (1, 36)
pre:  (100, 10)
num:  (437, 10)


## `StmtRawFilter`
This filter filters the data by the stmt column, which defines to which financial statement (BalanceSheet, CashFlow, IncomeStatement, ..) a data point belongs.

The stmt column is part of the pre_df, so only the pre_df is filtered by this filter.

In [4]:
from secfsdstools.e_filter.rawfiltering import StmtRawFilter

stmt_filter = StmtRawFilter(stmts=['BS', 'CF'])

filtered_databag = databag[stmt_filter]

# as expected, only the shape of the pre_df is different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (7280, 36)
pre:  (508458, 10)
num:  (3543392, 10)


## `ReportPeriodRawFilter`
A report always contains datapoints from previous years. For instance, in the BalanceSheet, IncomeStatement, and CashFlow statement also the values for the previous year is displayed.

Moreover, it is common in an annual report that certain datapoints are presented for several previous years.

You can use this filter, if you are only interested in datapoints that belongs to the "period" date of the report (defined in the period column of the sub_df).

The column ddate of the num_df does define the date of the datapoint. So this filter only applies to the num_df.

To demonstrate that, let us first have a look at all the ddates that are present in apple's 2022 10-K report.

In [9]:
from secfsdstools.e_filter.rawfiltering import AdshRawFilter

apple_10k_2022_adsh = "0000320193-22-000108"
adsh_filter = AdshRawFilter(adshs=[apple_10k_2022_adsh])

apple_10k_2022_databag = databag[adsh_filter]

# first, let's see for what period the report is
# The format is YYYYMMDD, so it is the 30th of September 2022
print(apple_10k_2022_databag.sub_df.period.tolist())

# Next, lets have a look how many different datapoint ddates are in the num_df and lets also count how many datapoints there are
print(apple_10k_2022_databag.num_df.ddate.value_counts())

[20220930]
20210930    169
20220930    165
20200930     95
20190930      8
Name: ddate, dtype: int64


As exepcted, most of the datapoints are from the period of the report (2022). But a significant number of datapoints is from the previous year (2021) and even the year before that (2020).

By applying this filter, only datapoints are filtered that have the same value for ddate as the period column of the report to which they belong.

In [11]:
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter

reportperiod_filter = ReportPeriodRawFilter()

filtered_databag = apple_10k_2022_databag[reportperiod_filter]

# let us check the number of datapoints per ddate after the filter. We expect to see just datapoints for one
print(filtered_databag.num_df.ddate.value_counts())

# as expected, only the shape of the num_df is different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

20220930    165
Name: ddate, dtype: int64
sub:  (1, 36)
pre:  (100, 10)
num:  (165, 10)


## `ReportPeriodAndPreviousPeriodRawFilter`
This filter is similar to the `ReportPeriodRawFilter` filter. Instead of just filtering the datapoints of the period of a report, it also filters the datapoints of the previous year.

It also operates only on the num_df.

In [12]:
from secfsdstools.e_filter.rawfiltering import ReportPeriodAndPreviousPeriodRawFilter

reportperiodandprevious_filter = ReportPeriodAndPreviousPeriodRawFilter()

filtered_databag = apple_10k_2022_databag[reportperiodandprevious_filter]

# let us check the number of datapoints per ddate after the filter. We expect to see just datapoints for two ddates
print(filtered_databag.num_df.ddate.value_counts())

# as expected, only the shape of the num_df is different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

20210930    169
20220930    165
Name: ddate, dtype: int64
sub:  (1, 36)
pre:  (100, 10)
num:  (334, 10)


## `TagRawFilter` 
This filter filters only datapoints for the defined tags.

it operates on the pre_df and the num_df.

In [13]:
from secfsdstools.e_filter.rawfiltering import TagRawFilter

tag_filter = TagRawFilter(tags=['Assets', 'AssetsCurrent', 'AssetsNoncurrent'])

filtered_databag = databag[tag_filter]

# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

print(filtered_databag.pre_df.tag.value_counts())

sub:  (7280, 36)
pre:  (14129, 10)
num:  (43723, 10)
Assets              7516
AssetsCurrent       6081
AssetsNoncurrent     532
Name: tag, dtype: int64


## `MainCoregRawFilter`
Holding companies often also report data of their subsidiaries in their main report. Datapoints of subsidiares contain the name of the subsidiary in the coreg column of the num_df dataframe.

A good example is "AMERICAN ELECTRIC POWER CO INC" (CIK 4904) which has about 7 subsidaries (e.g., Southwestern Electric Power Co, Indiana Michigan Power Co).

If you are not interested in the datapoints of subsidiaries, you can use this filter to remove datapoints of subsidiaries.

This filter operates only on the num_df dataframe.

In [14]:
from secfsdstools.e_filter.rawfiltering import AdshRawFilter

# let us select the 10q from American Electric Power Co Inc
americanelectric_10q_adsh="0000004904-22-000093"
americanelectric_10q_bag = databag[AdshRawFilter(adshs=[americanelectric_10q_adsh])]

# let's see what subsidiaries american electric has
print(americanelectric_10q_bag.num_df.coreg.unique())

print("sub: ", americanelectric_10q_bag.sub_df.shape)
print("pre: ", americanelectric_10q_bag.pre_df.shape)
print("num: ", americanelectric_10q_bag.num_df.shape)

['' 'OhioPowerCo' 'AEPTransmissionCo' 'IndianaMichiganPowerCo'
 'AEPTexasInc.' 'SouthwesternElectricPowerCo' 'PublicServiceCoOfOklahoma'
 'AppalachianPowerCo']
sub:  (1, 36)
pre:  (227, 10)
num:  (4796, 10)


In [16]:
from secfsdstools.e_filter.rawfiltering import MainCoregRawFilter

# now filter only the datapoints for the holding company
americanelectric_main_10q_bag_filtered = americanelectric_10q_bag[MainCoregRawFilter()]

# let's see what subsidiaries american electric have after the applying the filter. There should only be an empty string in the list
print(americanelectric_main_10q_bag_filtered.num_df.coreg.unique())


print("sub: ", americanelectric_main_10q_bag_filtered.sub_df.shape)
print("pre: ", americanelectric_main_10q_bag_filtered.pre_df.shape)
print("num: ", americanelectric_main_10q_bag_filtered.num_df.shape)

['']
sub:  (1, 36)
pre:  (227, 10)
num:  (2854, 10)


## `OfficialTagsOnlyRawFilter`
Sometimes companies use inofficial tags than the standard us-gaap tags defined by the xbrl taxonomy. In this cases, the version column contains the adsh number of the report, instead something like us-gaap/2022.

Especially if you want to compare reports from different companies, it makes sense to only analyze the official tags.

You can use this filter to do that.

It operates on the pre_df and the num_df dataframe.

In [17]:
# let's check how many different version tags there are in all the reports of a single quarter, and display the first 20
print(len(databag.num_df.version.unique()), databag.num_df.version.unique()[:20])

7090 ['us-gaap/2022' '0001822993-22-000032' '0001637207-22-000054'
 'us-gaap/2021' '0001764013-22-000140' '0001398344-22-022208'
 '0000107140-22-000098' '0000914208-22-000500' '0001493152-22-034057'
 '0001213900-22-072201' '0001061219-22-000027' '0001477932-22-009083'
 'ifrs/2021' '0001521951-22-000071' '0001193125-22-279123' 'ifrs/2022'
 'us-gaap-sup/2022q3' '0000924901-22-000019' '0001493152-22-031374'
 '0001683168-22-007745']


In [19]:
# it looks as if it is quite common to use inofficial tags as well, so it might definitely make sense to filter only the official tags
from secfsdstools.e_filter.rawfiltering import OfficialTagsOnlyRawFilter

# now filter only the datapoints for the holding company
officialtagsonly_filter = OfficialTagsOnlyRawFilter()

# let's check the number and values for the version field
filtered_databag = databag[officialtagsonly_filter]

print(len(filtered_databag.num_df.version.unique()), filtered_databag.num_df.version.unique()[:20])


# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

9 ['us-gaap/2022' 'us-gaap/2021' 'ifrs/2021' 'ifrs/2022'
 'us-gaap-sup/2022q3' 'srt/2022' 'srt-sup/2022q3' 'srt/2021' 'dei/2022']
sub:  (7280, 36)
pre:  (714187, 10)
num:  (3217041, 10)


## `USDOnlyRawFilter`
International companies often report datapoints also in other currencies than just USD.

And again, if we want to compare the report of different companies, we might be only interested in USD datapoints.

This filter operates on the uom column of the num_df dataframe and removes entries that are not USD.

In [20]:
# let's check how many different "units" there are in all the reports of a single quarter, and display the first 30
print(len(databag.num_df.uom.unique()), databag.num_df.uom.unique()[:30])

71 ['USD' 'shares' 'CAD' 'ZAR' 'pure' 'EUR' 'CNY' 'BRL' 'AUD' 'JPY' 'GBP'
 'MXN' 'Contract' 'ARS' 'HKD' 'Rate' 'CHF' 'Derivative' 'oz' 'vote' 'SEK'
 'Litecoin' 'DKK' 'ISK' 'NOK' 'Share' 'Contracts' 'NumberOfContracts'
 'Uniswap' 'Class']


In [22]:
from secfsdstools.e_filter.rawfiltering import USDOnlyRawFilter

# now filter only the datapoints for the holding company
usdonly_filter = USDOnlyRawFilter()

filtered_databag = databag[usdonly_filter]

# let's check if just USD is kept as currency
print(len(filtered_databag.num_df.uom.unique()), filtered_databag.num_df.uom.unique()[:30])

# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

39 ['USD' 'shares' 'pure' 'Contract' 'Rate' 'Derivative' 'oz' 'vote'
 'Litecoin' 'Share' 'Contracts' 'NumberOfContracts' 'Uniswap' 'Class'
 'Vote' 'BitcoinCash' 'Firm' 'contract' 'number_of_store' 'Ethereum'
 'investment' 'dividend' 'agreement' 'Equipment' 'Decimal' 'Store'
 'Bitcoin' 'Avalanche' 'Zec' 'Horizen']
sub:  (7280, 36)
pre:  (806371, 10)
num:  (3480956, 10)


## `NoSegmentInfoRawFilter`

In December of 2024, the SEC recreated all the datasets to include also segment information for datapoints. Segment information deliver additional information for a datapoint from different viewpoints or on different axis.

For instance, a report normally contains the revenues of a company. However, with additional datapoints that contain the appropriate segment information, the report could also provide additional information that may show revenue for different region and/or the revenue for different products. So while the main number is reported with out segment information does provide the overall revenue, additional datapoints with segment information (but for the same tag) could provide more a breakdown on several "axes", like region or product.

If you are not interested in a deeper breakdown of the data you can use the `NoSegmentInfoRawFilter` to remove all datapoints that contain 'segment' information.

In [26]:
# let's check how many different "segment" values are in all the reports of a single quarter, and display the first 30
print(len(databag.num_df.segments.unique()), databag.num_df.segments.unique()[:30])

153533 [''
 'BusinessSegments=KingDigitalEntertainment;ConsolidationItems=OperatingSegments;ProductOrService=OtherDistributionChannels;'
 'ClassOfStock=SeriesDPreferredStock;EquityComponents=PreferredStock;'
 'Geographical=US;ProductOrService=UltraDeepwaterFloaters;'
 'EquityComponents=AccumulatedOtherComprehensiveIncome;LegalEntity=AppalachianPowerCo;'
 'EquityComponents=AccumulatedNetInvestmentGainLossIncludingPortionAttributableToNoncontrollingInterest;ReclassificationOutOfAccumulatedOtherComprehensiveIncome=ReclassificationOutOfAccumulatedOtherComprehensiveIncome;'
 'Restatement=ScenarioPreviouslyReported;'
 'EquityComponents=AccumulatedDistributionsInExcessOfNetIncome;'
 'BusinessSegments=SpecialtySegment;'
 'ClassOfStock=ConvertiblePreferredStock;EquityComponents=PreferredStock;'
 'EquityComponents=SeriesAPreferredStocks;'
 'FairValueByFairValueHierarchyLevel=FairValueInputsLevel1;FairValueByMeasurementFrequency=FairValueMeasurementsRecurring;FinancialInstrument=ExchangeTradedOpt

In [29]:
from secfsdstools.e_filter.rawfiltering import NoSegmentInfoRawFilter

no_segment_filter = NoSegmentInfoRawFilter()

filtered_databag = databag[no_segment_filter]

# let us check the segment column again. as expected, there should only be datapoints with an emptystring in their segments column
print(len(filtered_databag.num_df.segments.unique()), filtered_databag.num_df.segments.unique()[:30])

# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

1 ['']
sub:  (7280, 36)
pre:  (806371, 10)
num:  (1749883, 10)


## Chaining Filters

If you want to compare reports between companies and different years, it might make sense to apply several filters.

For instance, you might only be interested in the datapoints of the actual period of a report (`ReportPeriodRawFilter`), you may only want to have datapoints of a holding company and don't care about the subsidiaries (`MainCoregRawFilter`), in order to have somewhat comparable data, you only want to have 'official' tags (`OfficialTagsOnlyRawFilter`), and finally, you only want datapoints in the USD (`USDOnlyRawFilter`).

There are different ways how you could implement a filter chain.

In [30]:
# using a lambda function
from typing import Callable
from secfsdstools.d_container.databagmodel import RawDataBag
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregRawFilter, OfficialTagsOnlyRawFilter, USDOnlyRawFilter

filter_chain: Callable[[RawDataBag], RawDataBag] = lambda x: x[ReportPeriodRawFilter()][MainCoregRawFilter()][OfficialTagsOnlyRawFilter()][USDOnlyRawFilter()]

filtered_databag = filter_chain(databag)

print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (7280, 36)
pre:  (714187, 10)
num:  (1297892, 10)


In [31]:
# using a pure function
from secfsdstools.d_container.databagmodel import RawDataBag
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregRawFilter, OfficialTagsOnlyRawFilter, USDOnlyRawFilter

def filter_func(databag: RawDataBag) -> RawDataBag:
    return databag[ReportPeriodRawFilter()][MainCoregRawFilter()][OfficialTagsOnlyRawFilter()][USDOnlyRawFilter()]

filtered_databag = filter_func(databag)

print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (7280, 36)
pre:  (714187, 10)
num:  (1297892, 10)


In [32]:
# implementing your own filter class
from secfsdstools.d_container.databagmodel import RawDataBag, FilterBase
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregRawFilter, OfficialTagsOnlyRawFilter, USDOnlyRawFilter


class FilterChain(FilterBase[RawDataBag]):
    
     def filter(self, databag: RawDataBag) -> RawDataBag:
        return databag[ReportPeriodRawFilter()][MainCoregRawFilter()][OfficialTagsOnlyRawFilter()][USDOnlyRawFilter()]

# implementing a filter class has the advantage that you can call the filter method on the databag, resp. that you can use the [] operator
filtered_databag = databag[FilterChain()]

print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (7280, 36)
pre:  (714187, 10)
num:  (1297892, 10)
